In [1]:
import xsimlab as xs
import numpy as np

In [2]:
@xs.process
class P:
    idx = xs.index(dims='idx')
    var = xs.variable(dims='idx', intent='out')
    
    def initialize(self):
        self.idx = np.arange(2)
    
    xs.runtime()
    def run_step(self):
        self.var = np.full(self.idx.shape, np.datetime64('2000-01-01'), dtype='datetime64[ns]')

In [3]:
model = xs.Model({'p': P})

In [4]:
ds_in = xs.create_setup(model, clocks={'main': range(2)}, output_vars={'p__var': 'main'})
ds_in

<xarray.Dataset>
Dimensions:  (main: 2)
Coordinates:
  * main     (main) int64 0 1
Data variables:
    *empty*

In [5]:
ds_out_no_scale = ds_in.xsimlab.run(model, decoding={'mask_and_scale': False})
ds_out_no_scale

<xarray.Dataset>
Dimensions:  (idx: 2, main: 2)
Coordinates:
  * idx      (idx) int64 0 1
  * main     (main) int64 0 1
Data variables:
    p__var   (main, idx) datetime64[ns] 2000-01-01 2000-01-01 ... 2000-01-01

In [6]:
# This error is clear now: '_FillValue' is in attrs
ds_out_no_scale.to_netcdf('nc0', engine='netcdf4')

TypeError: Invalid value for attr '_FillValue': numpy.datetime64('1970-01-01T00:00:00.000000000') must be a number, a string, an ndarray or a list/tuple of numbers/strings for serialization to netCDF files

In [7]:
ds_out_no_scale.p__var.attrs.pop('_FillValue')

numpy.datetime64('1970-01-01T00:00:00.000000000')

In [11]:
# Less clear: '_FillValue' removed from attrs
ds_out_no_scale.to_netcdf('nc0', engine='netcdf4')

ValueError: unsupported dtype for netCDF4 variable: datetime64[ns]

In [12]:
# Test with {'mask_and_scale': True}
# idx is now a float and first value nan ...
# Ok, why not, but netcdf still fails
ds_out_scale = ds_in.xsimlab.run(model, decoding={'mask_and_scale': True})
ds_out_scale

<xarray.Dataset>
Dimensions:  (idx: 2, main: 2)
Coordinates:
  * idx      (idx) float64 nan 1.0
  * main     (main) int64 0 1
Data variables:
    p__var   (main, idx) datetime64[ns] 2000-01-01 2000-01-01 ... 2000-01-01

In [13]:
ds_out_scale.to_netcdf('nc', engine='netcdf4')

TypeError: The DTypes <class 'numpy.dtype[datetime64]'> and <class 'numpy.dtype[int64]'> do not have a common DType. For example they cannot be stored in a single array unless the dtype is `object`.